## Read and plot near-real-time Wave Glider data

first cut by Tom, 10/18/2021  
Updated for IOP1, 10/9/2022

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + 'SMODE_'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-127,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,21)-2.5
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 41)
    levels = np.linspace(13,18,11)

    

Payload 2 Table 1 has met, ctd variables  
Payload 2 Table 2 has RDI variables

In [4]:
# List of WGs
input_list = ['WHOI-ASL22','WHOI-ASL32','SV3-1043','STOKES', 'PLANCK', 'PASCAL', 'KELVIN', 'CARSON']
url_prefix = 'http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/'
tab1_postfix = '_PLD2_TAB1.nc'
tab2_postfix = '_PLD2_TAB2.nc'
WG_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'PASCAL', 'KELVIN', 'CARSON']
outpath='../data/raw/WG_NRT/'

In [5]:
# For some reason, reading the files over the internet directly is not working well
# Download instead

n=0
file_list1 = []
file_list2 = []
for WG in WG_list:
    input_WG=input_list[n]
    outfile1 = outpath+input_WG+tab1_postfix
    outfile2 = outpath+input_WG+tab2_postfix
    url1 = url_prefix+input_WG+tab1_postfix
    file_data = requests.get(url1).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile1, "wb") as file:
        file.write(file_data)
    url2 = url_prefix+input_WG+tab2_postfix
    file_data = requests.get(url2).content
    # create the file in write binary mode, because the data we get from net is in binary
    with open(outfile2, "wb") as file:
        file.write(file_data)
    n=n+1
    print(url1)
    file_list1.append(outfile1)
    file_list2.append(outfile2)


http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/WHOI-ASL22_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/WHOI-ASL32_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/SV3-1043_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/STOKES_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/PLANCK_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/PASCAL_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/KELVIN_PLD2_TAB1.nc
http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/CARSON_PLD2_TAB1.nc


In [6]:
file_list1

['../data/raw/WG_NRT/WHOI-ASL22_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/WHOI-ASL32_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/SV3-1043_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/STOKES_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/PLANCK_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/PASCAL_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/KELVIN_PLD2_TAB1.nc',
 '../data/raw/WG_NRT/CARSON_PLD2_TAB1.nc']

In [7]:
# Read in Payload 2 Tables 1 and 2 from all WG
n=0
for WG in WG_list:
    input_WG=input_list[n]
    file1 = file_list1[n]
    file2 = file_list2[n]
    varstr = 'ds1_'+WG
    locals()[varstr]=xr.open_dataset(file1,decode_times=True)    
    varstr = 'ds2_'+WG
    locals()[varstr]=xr.open_dataset(file2,drop_variables=['z'],decode_times=True)
    n=n+1
    print(file1)

../data/raw/WG_NRT/WHOI-ASL22_PLD2_TAB1.nc
../data/raw/WG_NRT/WHOI-ASL32_PLD2_TAB1.nc
../data/raw/WG_NRT/SV3-1043_PLD2_TAB1.nc
../data/raw/WG_NRT/STOKES_PLD2_TAB1.nc
../data/raw/WG_NRT/PLANCK_PLD2_TAB1.nc
../data/raw/WG_NRT/PASCAL_PLD2_TAB1.nc
../data/raw/WG_NRT/KELVIN_PLD2_TAB1.nc
../data/raw/WG_NRT/CARSON_PLD2_TAB1.nc


In [8]:
# Now we can access these in a loop using syntax like:
eval('ds1_'+WG_list[7]).time

<xarray.DataArray 'time' (time: 1158)>
array(['2022-09-22T20:16:00.000000000', '2022-09-22T20:30:56.000000000',
       '2022-09-22T20:45:52.000000000', ..., '2022-10-10T06:00:32.000000000',
       '2022-10-10T06:15:28.000000000', '2022-10-10T06:30:24.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2022-09-22T20:16:00 ... 2022-10-10T06:30:24
Attributes:
    axis:           T
    long_name:      Time of waveglider measurement
    standard_name:  time

In [9]:
eval('ds1_'+WG_list[0])

<xarray.Dataset>
Dimensions:                      (time: 874)
Coordinates:
  * time                         (time) datetime64[ns] 2022-09-22T02:31:28 .....
Data variables: (12/71)
    year                         (time) float32 ...
    day                          (time) float32 ...
    hour                         (time) float32 ...
    minute                       (time) float32 ...
    sec                          (time) float32 ...
    RECORD                       (time) float32 ...
    ...                           ...
    sig_temperature_Avg          (time) float32 ...
    index_1hz_file               (time) float32 ...
    index_ins_file               (time) float32 ...
    index_psp_file               (time) float32 ...
    index_adcp_file              (time) float32 ...
    index_sig_file               (time) float32 ...
Attributes:
    Name:                 WHOI-ASL22_PLD2_TAB1
    wms_title:            WHOI-ASL22_PLD2_TAB1
    title:                S-MODE IOP1 2022 Waveglider WHOI-ASL22 real-time da...
    instrument:           Liquid Robotics SV3 Waveglider
    sea_name:             Pacific
    keywords:             Waveglider, S-MODE, Pacific Ocean, Temperature
    history:              2022-10-10 02:35Z Downloaded real-time data from WG...
    time_coverage_start:  2022-09-21T22:31:00
    time_coverage_end:    2022-10-10T02:31:00

In [10]:
p = 1
for WG in WG_list:
    ds = eval('ds1_'+WG)
    ds['uctd_psu_Avg']=gsw.conversions.SP_from_C(10*ds.uctd_cond_Avg, ds.uctd_temp_Avg, p)
    SA = gsw.conversions.SA_from_SP(ds.uctd_psu_Avg, 1,ds.longitude_1hz_Avg, ds.latitude_1hz_Avg)
    CT = gsw.conversions.CT_from_t(SA, ds.uctd_temp_Avg, p)
    ds['uctd_sigma0_Avg'] = gsw.density.sigma0(SA, CT)
    varstr = 'ds1_'+WG
    locals()[varstr]= ds

In [11]:
ds

<xarray.Dataset>
Dimensions:                      (time: 1158)
Coordinates:
  * time                         (time) datetime64[ns] 2022-09-22T20:16:00 .....
Data variables: (12/68)
    year                         (time) float32 ...
    day                          (time) float32 ...
    hour                         (time) float32 ...
    minute                       (time) float32 ...
    sec                          (time) float32 ...
    RECORD                       (time) float32 ...
    ...                           ...
    index_1hz_file               (time) float32 ...
    index_10hz_file              (time) float32 ...
    index_ADCP_file              (time) float32 ...
    index_WCTD_file              (time) float32 ...
    uctd_psu_Avg                 (time) float64 33.48 33.48 ... 32.56 32.61
    uctd_sigma0_Avg              (time) float64 24.64 24.64 ... 23.61 23.55
Attributes:
    Name:                 CARSON_PLD2_TAB1
    wms_title:            CARSON_PLD2_TAB1
    title:                S-MODE IOP1 2022 Waveglider CARSON real-time data -...
    instrument:           Liquid Robotics SV3 Waveglider
    sea_name:             Pacific
    keywords:             Waveglider, S-MODE, Pacific Ocean, Temperature
    history:              2022-10-10 02:35Z Downloaded real-time data from WG...
    time_coverage_start:  2022-09-22T16:16:00
    time_coverage_end:    2022-10-10T02:31:00

In [12]:
def plot_WG_SST(V,n,tmin):
    extent = [xmin, xmax, ymin, ymax]
    ds = eval('ds1_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    sst = ds2.uctd_temp_Avg.values.astype(np.ndarray)
    ax.set_title('WG SST',size = 10.)
    plt.scatter(ds2.longitude_1hz_Avg, ds2.latitude_1hz_Avg,s=5,c=sst, cmap=plt.get_cmap('turbo'),vmin=V[0],vmax=V[1],transform=ccrs.PlateCarree())
    
    

In [13]:
def plot_WG_sigma0(V,n,tmin):
    extent = [xmin, xmax, ymin, ymax]
    ds = eval('ds1_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    rho = ds2.uctd_sigma0_Avg.values.astype(np.ndarray)
    ax.set_title('$\sigma_0$',size = 10.)
    plt.scatter(ds2.longitude_1hz_Avg, ds2.latitude_1hz_Avg,s=5,c=rho, cmap=plt.get_cmap('turbo'),vmin=V[0],vmax=V[1],transform=ccrs.PlateCarree())
 

In [14]:
fig = plt.figure()
V = [14,18]
Vrho = [23,24.5]
tmin = np.datetime64('2022-10-05T00:00:00')
n = 0
extent = [xmin, xmax, ymin, ymax]
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
ax.set_extent(extent, crs=ccrs.PlateCarree())


for n in range(8):
        #plot_WG_time(n)
        #plot_WG_SST(V,n,tmin)
        plot_WG_sigma0(Vrho,n,tmin)


plt.colorbar(location = 'bottom')
ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')

gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
functions.plot_ops_area(ax,transform=ccrs.PlateCarree(),color='k');


{'transform': <cartopy.crs.PlateCarree object at 0x7f8b07251cc0>, 'color': 'k'}


In [44]:
def plot_WG_sigma0_lat(n,tmin):
    ds = eval('ds1_'+WG_list[n])
    ds2 = ds.where(ds.time>tmin)
    rho = ds2.uctd_sigma0_Avg.values.astype(np.ndarray)
    ax.set_title('$\sigma_0$',size = 10.)
    plt.scatter(rho,ds2.latitude_1hz_Avg,s=5,c=(ds2.time-tmin)*10**-9/(60*60*24), cmap=plt.get_cmap('turbo'))


In [46]:
fig = plt.figure()
V = [14,18]
Vrho = [23,24.5]
tmin = np.datetime64('2022-10-05T00:00:00')
for n in range(8):
        #plot_WG_time(n)
        #plot_WG_SST(V,n,tmin)
        plot_WG_sigma0_lat(n,tmin)

plt.colorbar(location = 'right')


In [27]:
foo = plt.xlim()

In [28]:
print(foo)

(0.0, 1.0)
